## Chap 2.6: Feature engineering


The process of feature engineering is of manipulating, transforming, selecting raw data into features that can be used in statistical analysis of prediction.

* statistical features
* temporal features
* spectral features (Fourier and Wavelet transforms)

We will download seismic data and work using the TSFEL package to extract features. Because TSFEL is not in the conda registry, we have to install it using pip.

In [1]:
%pip install tsfel

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import modules for seismic data and feature extraction
import obspy
import numpy as np
import tsfel

import obspy.clients.fdsn.client as fdsn
from obspy import UTCDateTime



ModuleNotFoundError: No module named 'obspy'

We will use a time series from a seismic record of the 10/7/2022 M4.4 Earthquake in Oregon. The earthquake had a moderate size but was felt by many citizens.

<div>
<img src="uw61886506_ciim_geo.jpeg" width="500"/>
</div>

We will first download data from the PNSN seismic station UW.MRIN.

In [ ]:
# Solution
network = 'UW'
station = 'MRIN'
channel = 'ENZ'# this channel gives a low frequency, 1Hz signal.
Tstart = UTCDateTime(year=2022, month=10, day=7)
Tend = UTCDateTime(year=2022, month=10, day=8)
fdsn_client = fdsn.Client('IRIS')
Z = fdsn_client.get_waveforms(network=network, station=station, location='--', channel=channel, starttime=Tstart, \
    endtime=Tend, attach_response=True)
Z.detrend()
Z.merge()
Z.plot()
T = np.linspace(0,(Tend-Tstart),Z[0].stats.npts)